# Farmer problem

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

In [1]:
using JuMP

In [2]:
# Open-source solvers
using GLPK
using HiGHS

Commercial solvers as `CPLEX` or `Gurobi` can also be used.

Among open-source solvers, we recommend HiGHS ((https://highs.info/), as suggested by the presentation done by Julian Hall at JuliaCon 2022: https://www.youtube.com/watch?v=ChKdTqUzXdY

# Average Scenario

We construct the model for the average scenario, using first the open-source HiGHS solver .

In [3]:
m = Model(HiGHS.Optimizer)
#m = Model(GLPK.Optimizer)

crops = ["Wheat", "Corn", "Sugar beets"]
ncrops = length(crops)

@variable(m, x[1:ncrops] >= 0)
@variable(m, y[1:2] >= 0)
@variable(m, w[1:4] >= 0)

costSeeding = [150, 230, 260]
costs = [238, 210]
prices = [170, 150, 36, 10]

4-element Vector{Int64}:
 170
 150
  36
  10

We can check that `m` is a JuMP model by checking its type.

In [4]:
typeof(m)

Model

In [5]:
typeof(x)

Vector{VariableRef} (alias for Array{VariableRef, 1})

We now add the constraints.

In [6]:
@constraint(m, surface, sum(x[i] for i=1:ncrops) <= 500)

surface : x[1] + x[2] + x[3] <= 500.0

In [7]:
@constraint(m, wheatNeeds, 2.5x[1]+y[1]-w[1] >= 200)

wheatNeeds : 2.5 x[1] + y[1] - w[1] >= 200.0

In [8]:
@constraint(m, cornNeeds, 3x[2]+y[2]-w[2] >= 240)

cornNeeds : 3 x[2] + y[2] - w[2] >= 240.0

In [9]:
@constraint(m, beetsProd, w[3]+w[4] <= 20x[3])

beetsProd : -20 x[3] + w[3] + w[4] <= 0.0

In [10]:
@constraint(m, beetsQuota, w[3] <= 6000)

beetsQuota : w[3] <= 6000.0

Let's define the objective function. 

In [11]:
@objective(m, Min, sum(costSeeding[i]*x[i] for i=1:ncrops) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))

150 x[1] + 230 x[2] + 260 x[3] + 238 y[1] + 210 y[2] - 170 w[1] - 150 w[2] - 36 w[3] - 10 w[4]

The complete model can be displayed with the command `print`.

In [12]:
print(m)

More information can be obtained with the method `show`:

In [13]:
show(m)

A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y

We now solve it.

In [14]:
optimize!(m)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
HiGHS run time      :          0.01


In [15]:
print("Solution: $(value.(x))")

Solution: [120.0, 80.0, 300.0]

In [16]:
? value

search: value values set_value fix_value has_values start_value callback_value



```
value(con_ref::ConstraintRef; result::Int = 1)
```

Return the primal value of constraint `con_ref` associated with result index `result` of the most-recent solution returned by the solver.

That is, if `con_ref` is the reference of a constraint `func`-in-`set`, it returns the value of `func` evaluated at the value of the variables (given by [`value(::VariableRef)`](@ref)).

Use [`has_values`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

## Note

For scalar constraints, the constant is moved to the `set` so it is not taken into account in the primal value of the constraint. For instance, the constraint `@constraint(model, 2x + 3y + 1 == 5)` is transformed into `2x + 3y`-in-`MOI.EqualTo(4)` so the value returned by this function is the evaluation of `2x + 3y`. ```

---

```
value(var_value::Function, con_ref::ConstraintRef)
```

Evaluate the primal value of the constraint `con_ref` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::VariableRef; result = 1)
```

Return the value of variable `v` associated with result index `result` of the most-recent returned by the solver.

Use [`has_values`](@ref) to check if a result exists before asking for values.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, v::VariableRef)
```

Evaluate the value of the variable `v` as `var_value(v)`.

---

```
value(var_value::Function, ex::GenericAffExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericAffExpr; result::Int = 1)
```

Return the value of the `GenericAffExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::GenericQuadExpr)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericQuadExpr; result::Int = 1)
```

Return the value of the `GenericQuadExpr` `v` associated with result index `result` of the most-recent solution returned by the solver.

Replaces `getvalue` for most use cases.

See also: [`result_count`](@ref).

---

```
value(p::NonlinearParameter)
```

Return the current value stored in the nonlinear parameter `p`.

# Example

```jldoctest; setup=:(using JuMP)
model = Model()
@NLparameter(model, p == 10)
value(p)

# output
10.0
```

---

```
value(ex::NonlinearExpression; result::Int = 1)
```

Return the value of the `NonlinearExpression` `ex` associated with result index `result` of the most-recent solution returned by the solver.

Replaces `getvalue` for most use cases.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, ex::NonlinearExpression)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(c::NonlinearConstraintRef; result::Int = 1)
```

Return the value of the `NonlinearConstraintRef` `c` associated with result index `result` of the most-recent solution returned by the solver.

See also: [`result_count`](@ref).

---

```
value(var_value::Function, c::NonlinearConstraintRef)
```

Evaluate `c` using `var_value(v)` as the value for each variable `v`.


In [17]:
methods(value)

# 17 methods for generic function "value" from JuMP:
  [1] value(::MutableArithmetics.Zero)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1337
  [2] value(ex::NonlinearExpression; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:328
  [3] value(p::NonlinearParameter)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:235
  [4] value(v::VariableRef; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1031
  [5] value(c::NonlinearConstraintRef; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:509
  [6] value(con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex}; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\constraints.jl:900
  [7] value(ex::GenericQuadExpr; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\quad_expr.jl:702
  [8] value(Q::LinearAlgebra.Symmetric{V, Matrix{V}}) where V<:AbstractVariableRef
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\sd.jl:346
  [9] value(::AbstractArray{<:AbstractJuMPScalar})
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1330
 [10] value(var_value::Function, c::NonlinearConstraintRef)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:520
 [11] value(var_value::Function, ex::NonlinearExpression)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\nlp.jl:363
 [12] value(var_value::Function, ex::GenericQuadExpr{CoefType, VarType}) where {CoefType, VarType}
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\quad_expr.jl:676
 [13] value(var_value::Function, ex::GenericAffExpr{T, V}) where {T, V}
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\aff_expr.jl:330
 [14] value(var_value::Function, con_ref::ConstraintRef{<:AbstractModel, <:MathOptInterface.ConstraintIndex})
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\constraints.jl:913
 [15] value(var_value::Function, v::VariableRef)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1040
 [16] value(x::Number)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\variables.jl:1338
 [17] value(a::GenericAffExpr; result)
     @ C:\Users\slash\.julia\packages\JuMP\60Bnj\src\aff_expr.jl:570

In [18]:
value(surface)

500.0

We will summarize the solution approach using a Julia function.

In [19]:
function farmer(factor::Float64 = 1.0, m::Model = Model(HiGHS.Optimizer))
    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m, x[1:ncrops] >= 0)
    @variable(m, y[1:2] >= 0)
    @variable(m, w[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = factor*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNeeds, returns[1]x[1]+y[1]-w[1] >= 200)
    @constraint(m, cornNeeds, returns[2]x[2]+y[2]-w[2] >= 240)
    @constraint(m, beetsProd, w[3]+w[4] <= returns[3]x[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)

    # println(value.(x))

    return m, status
end

farmer (generic function with 3 methods)

In [20]:
methods(farmer)

# 3 methods for generic function "farmer" from Main:
 [1] farmer(factor::Float64, m::Model)
     @ In[19]:1
 [2] farmer(factor::Float64)
     @ In[19]:1
 [3] farmer()
     @ In[19]:1

In [21]:
maverage, status = farmer()

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.1860000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.1860000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [22]:
maverage

A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y

In [23]:
print(maverage)

In [24]:
objective_value(maverage)

-118600.0

We can recover the decision variables with the method `all_variables`:

In [25]:
variables = all_variables(maverage)

9-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 y[1]
 y[2]
 w[1]
 w[2]
 w[3]
 w[4]

We can thus get the first-stage decisions.

In [26]:
value.(variables[1:3])

3-element Vector{Float64}:
 120.0
  80.0
 300.0

## Good scenario

In [27]:
mgood, status = farmer(1.2)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.3999740210e+02 Ph1: 2(2); Du: 2(639.997) 0s
          4    -1.6766666667e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 4
Objective value     : -1.6766666667e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [28]:
mgood

A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y

In [29]:
objective_value(mgood)

-167666.6666666667

## Bad scenario

In [30]:
mbad, status = farmer(0.8)
#mbad, status = farmer(0.8, Model(solver = GurobiSolver()))

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
4 rows, 9 cols, 12 nonzeros
3 rows, 8 cols, 10 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-2); columns 7(-2); elements 9(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.6999813050e+02 Ph1: 2(2); Du: 2(469.998) 0s
          5    -5.9950000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     : -5.9950000000e+04
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, surface, w, wheatNeeds, x, y, nothing)

In [31]:
objective_value(mbad)

-59950.0

## Stochastic program - extensive form

We have to combine the three scenarios. We will assume that each one has a probability equal to 1/3.

In [32]:
n = 3
p = [1/n for i = 1:n]

3-element Vector{Float64}:
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333

We will start with a naive implementation.

In [34]:
function farmerStoch()
    m = Model(HiGHS.Optimizer)

    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)

    @variable(m,ya[1:2] >= 0)
    @variable(m,wa[1:4] >= 0)

    @variable(m,yg[1:2] >= 0)
    @variable(m,wg[1:4] >= 0)

    @variable(m,yb[1:2] >= 0)
    @variable(m,wb[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returnsA = [2.5, 3, 20]
    returnsG = 1.2*returnsA
    returnsB = 0.8*returnsA

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNA, returnsA[1]x[1]+ya[1]-wa[1] >= 200)
    @constraint(m, cornNA, returnsA[2]x[2]+ya[2]-wa[2] >= 240)
    @constraint(m, beetsPA, wa[3]+wa[4] <= returnsA[3]x[3])
    @constraint(m, beetsQA, wa[3] <= 6000)

    @constraint(m, wheatNG, returnsG[1]x[1]+yg[1]-wg[1] >= 200)
    @constraint(m, cornNG, returnsG[2]x[2]+yg[2]-wg[2] >= 240)
    @constraint(m, beetsPG, wg[3]+wg[4] <= returnsG[3]x[3])
    @constraint(m, beetsQG, wg[3] <= 6000)

    @constraint(m, wheatNB, returnsB[1]x[1]+yb[1]-wb[1] >= 200)
    @constraint(m, cornNB, returnsB[2]x[2]+yb[2]-wb[2] >= 240)
    @constraint(m, beetsPB, wb[3]+wb[4] <= returnsB[3]x[3])
    @constraint(m, beetsQB, wb[3] <= 6000)

    @objective(m, Min, sum(costSeeding[i]*x[i] for i=1:3)
        + 1/3*sum(costs[i]*ya[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wa[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yg[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wg[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yb[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wb[i] for i=1:length(prices)))
    
    status = optimize!(m)

    return m, status
end

farmerStoch (generic function with 1 method)

In [35]:
mstoch, status = farmerStoch()

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 21 cols, 30 nonzeros
9 rows, 20 cols, 30 nonzeros
Presolve : Reductions: rows 9(-4); columns 20(-1); elements 30(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.1831206476e+02 Ph1: 8(8); Du: 8(318.312) 0s
         15    -1.0839000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 15
Objective value     : -1.0839000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 21
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 21 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsPA, beetsPB, beetsPG, beetsQA, beetsQB, beetsQG, cornNA, cornNB, cornNG, surface, wa, wb, wg, wheatNA, wheatNB, wheatNG, x, ya, yb, yg, nothing)

In [36]:
print(mstoch)

In [37]:
valStoch = objective_value(mstoch)

-108390.0

## Expected value of the perfect information

In [38]:
values = [objective_value(maverage), objective_value(mgood), objective_value(mbad)]

3-element Vector{Float64}:
 -118600.0
 -167666.6666666667
  -59950.0

Let's compute the expected revenue under perfect information, simply obtained as the dot product between the probabilities vector and the optimal values vector.

In [39]:
valPerfect = p'*values

-115405.55555555555

In [40]:
using LinearAlgebra

dot(p, values)

-115405.55555555555

In [41]:
methods(dot)

# 54 methods for generic function "dot" from LinearAlgebra:
  [1] dot(x::BitVector, y::BitVector)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\bitarray.jl:3
  [2] dot(x::Number, y::Number)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\generic.jl:876
  [3] dot(x::Vector{T}, rx::AbstractRange{TI}, y::Vector{T}, ry::AbstractRange{TI}) where {T<:Union{ComplexF64, ComplexF32}, TI<:Integer}
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:31
  [4] dot(x::Vector{T}, rx::AbstractRange{TI}, y::Vector{T}, ry::AbstractRange{TI}) where {T<:Union{Float32, Float64}, TI<:Integer}
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:18
  [5] dot(A::Diagonal, B::Diagonal)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\diagonal.jl:839
  [6] dot(D::Diagonal, B::AbstractMatrix)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\diagonal.jl:840
  [7] dot(A::AbstractMatrix, B::Diagonal)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\diagonal.jl:845
  [8] dot(J::UniformScaling, A::AbstractMatrix)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\uniformscaling.jl:423
  [9] dot(A::SparseArrays.AbstractSparseMatrixCSC{T1, S1}, B::SparseArrays.AbstractSparseMatrixCSC{T2, S2}) where {T1, T2, S1, S2}
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\SparseArrays\src\linalg.jl:316
 [10] dot(A::SparseArrays.AbstractSparseMatrixCSC, B::Union{SubArray{<:Any, 2, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, Adjoint{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, Hermitian{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, LowerTriangular{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, Symmetric{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, Transpose{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, UnitLowerTriangular{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, UnitUpperTriangular{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, UpperHessenberg{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, UpperTriangular{<:Any, Union{StridedMatrix, BitMatrix, SparseArrays.AbstractSparseMatrix}}, StridedMatrix, BitMatrix})
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\SparseArrays\src\linalg.jl:426
 [11] dot(A::Symmetric, B::Symmetric)
     @ C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\LinearAlgebra\src\symmetric.jl:457
 [12] dot(x::SparseArrays.SparseVector, A::SparseArrays.AbstractSparseMatrixCSC, y::SparseArrays.SparseVector)
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\SparseArrays\src\linalg.jl:369
 [13] dot(x::SparseArrays.SparseVector, A::Union{Hermitian{var"#s390", var"#s389"}, Hermitian{Complex{var"#s390"}, var"#s389"}, Symmetric{var"#s390", var"#s389"}} where {var"#s390", var"#s389"<:SparseArrays.AbstractSparseMatrixCSC}, y::SparseArrays.SparseVector)
     @ SparseArrays C:\Users\slash\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\SparseArrays\src\linalg.jl:947
 [14] dot(x::AbstractVector, A::Union{Hermitian{var"#s994", var"#s128"}, Symmetric{var"#s994", var"#s128"}, Symmetric{Complex{var"#s994"}, var"#s128"}} where {var"#s994"<:Real, var"#s128"<:Diagonal}, y::AbstractVect

Expected value of perfect information:

In [43]:
valPerfect - valStoch

-7015.5555555555475

## Value of the stochastic solution

We have to fix the first stage decision. We define the second stage problem.

In [45]:
function secondStage(x, factor::Float64 = 1.0)
    m = Model(HiGHS.Optimizer)
    
    @variable(m,y[1:2] >= 0)   # purchases
    @variable(m,w[1:4] >= 0)   # sales

    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = (factor*[2.5, 3, 20]).*x
    z = [200 - returns[1], 240 - returns[2], returns[3]]
    
    @constraint(m, wheatNeeds, y[1]-w[1] >= z[1])
    @constraint(m, cornNeeds, y[2]-w[2] >= z[2])
    @constraint(m, beetsProd, w[3]+w[4] <= z[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min, sum(costs[i]*y[i] for i = 1:length(costs)) - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)
    
    return m, status
end

secondStage (generic function with 2 methods)

In [46]:
x = [120, 80, 300]

3-element Vector{Int64}:
 120
  80
 300

In [47]:
msecond, status = secondStage(x)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 3 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Using EKK primal simplex solver
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.3300006374e+05 Pr: 0(0); Du: 1(10) 0s
          1    -2.3300000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 1
Objective value     : -2.3300000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, w, wheatNeeds, y, nothing)

In [48]:
print(msecond)

In [49]:
vaverage = objective_value(msecond)

-233000.0

In [50]:
msecond, status = secondStage(x, 1.2)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3 rows, 6 cols, 6 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -2.6240000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, w, wheatNeeds, y, nothing)

In [51]:
vgood = objective_value(msecond)

-262400.0

In [52]:
msecond, status = secondStage(x, 0.8)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 3 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-6); elements 0(-7) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -1.6952000000e+05
HiGHS run time      :          0.00


(A JuMP Model
Minimization problem with:
Variables: 6
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: HiGHS
Names registered in the model: beetsProd, beetsQuota, cornNeeds, w, wheatNeeds, y, nothing)

In [53]:
vbad = objective_value(msecond)

-169520.0

Expected revenue

In [54]:
er = costSeeding'*x+p'*[vaverage, vgood, vbad]

-107239.99999999997

Value of the stochastic solution

In [55]:
vss = valStoch - er

-1150.000000000029